In [1]:
import openml
import numpy as np
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from lrcb.representations.finite_representations import LinearRepresentation, normalize_param, rank, hls_rank, hls_lambda

### Fetch classification datasets

Turn OpenML classification problem into contextual bandit

Datasets from Bietti, Alberto, Alekh Agarwal, and John Langford. "A contextual bandit bake-off." arXiv preprint arXiv:1802.04064 (2018). https://github.com/albietz/cb_bakeoff

These are the OpenML ids of the original datasets selected by Bietti et al.

In [2]:
ids = [3, 6, 8, 10, 11, 12, 14, 16, 18, 20, 21, 22, 23, 26, 28, 30, 31, 32, 36, 37, 39, 40, 41, 43, 44, 46, 48, 50, 53, 54, 59, 60, 61, 62, 150, 151, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 180, 181, 182, 183, 184, 187, 189, 197, 209, 223, 227, 273, 275, 276, 277, 278, 279, 285, 287, 292, 293, 294, 298, 300, 307, 310, 312, 313, 329, 333, 334, 335, 336, 337, 338, 339, 343, 346, 351, 354, 357, 375, 377, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 444, 446, 448, 450, 457, 458, 459, 461, 462, 463, 464, 465, 467, 468, 469, 472, 475, 476, 477, 478, 479, 480, 554, 679, 682, 683, 685, 694, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 758, 759, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 782, 783, 784, 785, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 799, 800, 801, 803, 804, 805, 806, 807, 808, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 832, 833, 834, 835, 836, 837, 838, 841, 843, 845, 846, 847, 848, 849, 850, 851, 853, 855, 857, 859, 860, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 884, 885, 886, 888, 891, 892, 893, 894, 895, 896, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 931, 932, 933, 934, 935, 936, 937, 938, 941, 942, 943, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 958, 959, 962, 964, 965, 969, 970, 971, 973, 974, 976, 977, 978, 979, 980, 983, 987, 988, 991, 994, 995, 996, 997, 1004, 1005, 1006, 1009, 1011, 1012, 1013, 1014, 1015, 1016, 1019, 1020, 1021, 1022, 1025, 1026, 1036, 1038, 1040, 1041, 1043, 1044, 1045, 1046, 1048, 1049, 1050, 1054, 1055, 1056, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1071, 1073, 1075, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1100, 1104, 1106, 1107, 1110, 1113, 1115, 1116, 1117, 1120, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1133, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153, 1154, 1155, 1156, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1164, 1165, 1166, 1169, 1216, 1217, 1218, 1233, 1235, 1236, 1237, 1238, 1241, 1242, 1412, 1413, 1441, 1442, 1443, 1444, 1449, 1451, 1453, 1454, 1455, 1457, 1459, 1460, 1464, 1467, 1470, 1471, 1472, 1473, 1475, 1481, 1482, 1483, 1486, 1487, 1488, 1489, 1496, 1498, 1590]
len(ids)

525

In [3]:
def get_dataset(id):
    ds = openml.datasets.get_dataset(id)
    X, y, _, _ = ds.get_data(target=ds.default_target_attribute)
    
    return X, y

Keep datasets with only real-valued features

In [4]:
"""
selected_ids = []
for i in ids:
    X, y = get_dataset(i)
    if np.all(X.dtypes=='float64'):
        numeric_ids.append(i)
"""

"\nselected_ids = []\nfor i in ids:\n    X, y = get_dataset(i)\n    if np.all(X.dtypes=='float64'):\n        numeric_ids.append(i)\n"

In [5]:
selected_ids = [11,14,16,18,22,28,32,36,37,39,40,41,53,54,59,60,61,153,156,157,158,159,160,161,162,181,182,189,197,223,227,275,276,277,278,279,287,294,298,300,310,377,457,464,467,468,476,554,713,715,716,717,718,721,722,723,725,726,727,728,729,730,731,732,733,734,735,736,737,740,742,743,744,746,749,750,751,752,753,754,755,756,758,759,761,762,763,766,768,769,770,772,773,774,775,776,777,778,779,780,782,783,785,787,788,789,791,792,793,794,795,797,799,800,801,803,805,806,807,808,812,813,814,815,816,817,819,820,821,822,823,824,827,828,829,830,832,833,834,837,838,841,843,845,846,847,849,850,851,855,857,859,860,863,866,868,869,870,871,873,874,876,877,878,879,880,882,884,885,886,888,892,893,894,895,896,900,901,903,904,905,906,907,908,909,910,911,912,913,914,916,917,918,919,920,922,924,925,926,927,929,931,932,933,935,936,937,943,946,948,952,958,962,969,970,971,973,974,976,977,978,979,980,994,995,996,997,1004,1005,1011,1015,1019,1020,1021,1043,1045,1046,1048,1049,1050,1054,1056,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1071,1073,1075,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1104,1106,1107,1117,1120,1122,1123,1124,1125,1126,1127,1128,1129,1130,1131,1132,1133,1135,1136,1137,1138,1139,1140,1141,1142,1143,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1155,1156,1157,1158,1159,1160,1161,1162,1163,1164,1165,1166,1216,1217,1218,1233,1413,1441,1442,1443,1444,1449,1451,1453,1454,1457,1459,1460,1464,1467,1471,1473,1475,1482,1487,1488,1489,1496]
len(selected_ids)

343

### Turn classification problem into contextual bandit
Contexts (x) are the original feature vectors of the problem (standardized)

Actions (a) are class labels

We fit a NN to predict p(a|x) given x concatenated to the one-hot encoding of a, using cross-entropy loss

Training data are the original data concatenated to each action encoding. Targets are 1 for correct class and 0 otherwise

The output unit of the NN is a sigmoid. The second-to-last units are our new context-action features. The score fed to the sigmoid is our reward (new ground truth). The last layer of weights are the "unknown" parameters of our linear reward model.

In [6]:
def standardize(X):
    scaler = preprocessing.StandardScaler().fit(X)
    return scaler.transform(X)

In [7]:
def representation_dataset(X,y):
    X = standardize(X)
    n_samples = X.shape[0]
    n_features = X.shape[1]
    assert len(y) == n_samples
    classes = y.values.unique()
    n_classes = len(classes)
    
    new_X = np.zeros((n_samples*n_classes, n_features+n_classes))
    new_y = np.zeros(n_samples*n_classes)
    
    for i in range(n_samples):
        for j in range(n_classes):
            one_hot = np.zeros(n_classes)
            one_hot[j] = 1.
            new_X[i*n_classes + j] = np.concatenate((X[i], one_hot))
            new_y[i*n_classes + j] = 1. if y[i] == classes[j] else 0.
    return new_X, new_y

In [8]:
def build_model(net, X, n_contexts, n_actions):
    
    # Build features
    hidden_layer_sizes = list(net.hidden_layer_sizes)

    layer_units = [X.shape[1]] + hidden_layer_sizes + [1]
    activations = [X]
    for i in range(net.n_layers_ - 1):
        activations.append(np.empty((X.shape[0], layer_units[i + 1])))

    net._forward_pass(activations)
    y_pred = activations[-1]

    # get weights
    last_w = net.coefs_[-1]
    bias = np.array(net.intercepts_[-1]).reshape((1, 1))
    last_w = np.concatenate([last_w, bias])

    # get last-layer features
    last_feat = np.array(activations[-2], dtype=np.float32)
    last_feat = np.concatenate([last_feat, np.ones((X.shape[0], 1))], axis=1)

    # get prediction
    pred = last_feat.dot(last_w)

    # get feature matrix
    d = hidden_layer_sizes[-1] + 1
    phi = np.empty((n_contexts, n_actions, d), dtype=np.float32)
    idx = 0
    for t in range(n_contexts):
        for z in range(n_actions):
            phi[t, z, :] = last_feat[idx, :] 
            idx += 1
    assert idx == last_feat.shape[0]

    # get param
    theta = np.array(last_w, dtype=np.float32).squeeze()

    return phi, theta

In [9]:
def learn_representation(X, y, test_size=0.25, hidden=(32,32), max_iter=500, normalize=True):
    n_contexts = X.shape[0]
    n_actions = len(y.values.unique())
    
    #Preprocess dataset 
    X_rep, y_rep = representation_dataset(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_rep, y_rep, test_size=test_size)
    
    #Fit NN
    net = MLPClassifier(hidden_layer_sizes=hidden, max_iter=max_iter, verbose=False)
    net.fit(X_train, y_train)
    score = net.score(X_test, y_test)
    
    #Build representation
    phi, theta = build_model(net, X_rep, n_contexts, n_actions)
    rep = LinearRepresentation(phi, theta)
    if normalize:
        rep = normalize_param(rep)
    return rep, score

In [42]:
test_size=0.25
hidden=(32,32)
max_iter=1000
normalize=True #normalize parameters of linear combination
tol = 1e-12

for i in selected_ids[0:10]:
    X, y = get_dataset(i)
    rep, score = learn_representation(X, y, test_size, hidden, max_iter, normalize)
    print("%d: accuracy=%f, rank=%d, hls_rank=%d, hls_lambda=%f" % (i, score, rank(rep,tol), hls_rank(rep,tol), hls_lambda(rep)))

11: accuracy=0.970149, rank=33, hls_rank=33, hls_lambda=0.019287
14: accuracy=0.946400, rank=33, hls_rank=33, hls_lambda=2.092236
16: accuracy=0.985400, rank=33, hls_rank=33, hls_lambda=0.002045
18: accuracy=0.953400, rank=32, hls_rank=31, hls_lambda=0.000000
22: accuracy=0.959400, rank=33, hls_rank=33, hls_lambda=1.502369
28: accuracy=0.993310, rank=33, hls_rank=33, hls_lambda=0.827592
32: accuracy=0.998071, rank=33, hls_rank=33, hls_lambda=0.545113
36: accuracy=0.993074, rank=33, hls_rank=32, hls_lambda=0.000000


/home/matteo/miniconda2/envs/robot/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


37: accuracy=0.718750, rank=33, hls_rank=33, hls_lambda=1.458927
39: accuracy=0.965774, rank=33, hls_rank=32, hls_lambda=0.000000
